In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
from datetime import datetime, date
import pyarrow as pa

passwords = "Projeto_Passwords/arquivo_base/passwords.csv"
password_classifier = "Projeto_Passwords/transformações/password_classifier.csv"
passwords_formated_data = "Projeto_Passwords/transformações/passwords_formated_date.csv"
# passwords_classifier = "Projeto_Passwords/transformações/passwords_classifier.csv"

In [2]:
caminho_bases = pathlib.Path('dataset_analysis')
df = pd.read_csv(passwords, low_memory=False)
# df_formated_data = pd.read_csv('/Users/fabioleite/Projetos2022_1/annaCaroline/Projeto-LEDA/data/processedData/processedDatapasswords_formated_data.csv', low_memory=False)
# df_ranked = pd.read_csv('/Users/fabioleite/Projetos2022_1/annaCaroline/Projeto-LEDA/data/processedData/passwords_classifier.csv', low_memory=False)
# df_password_classifier = pd.read_csv(password_classifier, low_memory=False) 
df_formated_data = pd.read_csv(passwords_formated_data, low_memory=False) #/passwords_formated_data
df_ranked = pd.read_csv(password_classifier, low_memory=False)
df_password_classifier = pd.read_csv(password_classifier, low_memory=False, header='infer')


FileNotFoundError: [Errno 2] No such file or directory: 'Projeto_Passwords/transformações/passwords_formated_date.csv'

# T1 Transformation

4. Transformar data para o formato a seguir DD/MM/AAAA
* Gerar um arquivo chamado passwords_formated_data.csv

In [ ]:


df_datas_formatadas_origem = df['data']
datas_formatadas_aluno = df_formated_data['date']
# df_formated_data = df_formated_data['data']
print(f"Datas vazias no df original: {df_datas_formatadas_origem.isnull().sum()}")
print(f"Datas vazias no df formatado: {datas_formatadas_aluno.isnull().sum()}")
# df_formated_data['data_formatada'] = pd.to_datetime(datas_formatadas_aluno, format="%d/%m/%Y", errors='raise')

pd.testing.assert_series_equal(df_datas_formatadas_origem, datas_formatadas_aluno, check_names=False)

# Verifica datas formatadas

In [ ]:
df['data'] = pd.to_datetime(df['data'])
df['data_formatada'] = df['data'].dt.strftime('%d/%m/%Y %H:%M:%S')
pd.testing.assert_series_equal(df['data_formatada'] , datas_formatadas_aluno, check_names=False)

# Classificação

4. Filtrar senha pela categória Boa e Muito Boa.
* Gerar um arquivo chamado passwords_classifier.csv

In [ ]:
import re

def classify_password(password: str) -> str:
        # Expressão regular para letras (maiúsculas e minúsculas), números e caracteres especiais
        result = ""
        
        
        if ( (re.search(r'[A-Za-z]', password) and re.search(r'[0-9]', password)) or 
                        (re.search(r'[A-Za-z]', password) and re.search(r'[\W_]', password)) or 
                        (re.search(r'[0-9]', password) and re.search(r'[\W_]', password)) ):
                if len(password) == 6: 
                        result = "Fraca"
                elif len(password) <= 5:
                        result = "Ruim"
        
        password_pattern = "^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*\W).{4,}$"
        pat = re.compile(password_pattern)

        if bool(re.search(pat, password)):
                
                if len(password) <= 7: 
                        result = "Boa"
                elif len(password) >= 8:
                        result = "Muito Boa"

        return result

to_print = (classify_password('Abbb11'))
print(to_print)
# Função para verificar se a senha contém apenas letras, números e caracteres especiais
# is_valid_password = lambda pwd: bool(re.match(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[!@#$%^&*()_+=\-{}:;"\'<>,.?/\\|~`]).+$', pwd))

# Função lambda para classificar a senha
# classify_password = lambda pwd: 'Boa' if len(pwd) <= 7 and is_valid_password(pwd) else ''

In [ ]:
# Função para verificar se a senha contém apenas letras, números e caracteres especiais
# password_muito_boa = lambda pwd: bool(re.match(r'^(?=.[a-z])(?=.[A-Z])(?=.*[\d\W]).{8,}$', pwd))

# Função lambda para classificar a senha
# classify_password_muito_boa = lambda pwd: 'Muito Boa' if bool(re.match(r'^(?=.[a-z])(?=.[A-Z])(?=.*[\d\W]).{8,}$', pwd)) else ''
# classify_password_muito_boa = lambda pwd, length, cls: 'Muito Boa' if length >= 8 else ''

In [ ]:
print("tamanho do df original:  {:,}".format(len(df)))
# print("tamanho do df_ranked: {:,}".format(df_ranked.size))

print("----------------------------------------------------------")
#Pega tanto as senhas boas quanto as muito boas
df_self_ranked = df

df_self_ranked['classificacao'] = df_self_ranked.password.apply(lambda x: classify_password(str(x)))
# df_self_ranked['classificacao'] = df_self_ranked.apply(lambda x: classify_password_muito_boa(str(x.password), x.length, x.classificacao), axis=1)

df_senhas_Boas = df_self_ranked[df_self_ranked['classificacao'].str.startswith("Boa")]
df_senhas_MuitoBoas = df_self_ranked[df_self_ranked['classificacao'].str.startswith("Muito Boa")]
df_senhas_Fraca = df_self_ranked[df_self_ranked['classificacao'].str.startswith("Fraca")]
df_senhas_Ruim = df_self_ranked[df_self_ranked['classificacao'].str.startswith("Ruim")]
# display(df_senhas_Boas_e_MTB)

print("Num senhas Boas no aquivo base: {:,}".format(len(df_senhas_Boas)))
print("Num senhas Muito Boas no arquivo base: {:,}".format(len(df_senhas_MuitoBoas)))
print("Num senhas Fracas no arquivo base: {:,}".format(len(df_senhas_Fraca)))
print("Num senhas Ruim no arquivo base: {:,}".format(len(df_senhas_Ruim)))
print("----------------------------------------------------------")


# df_passwords_classifier_Boas = df_passwords_classifier[df_passwords_classifier['class'].str.startswith("Boa")]
df_password_classifier_Boas = (df_password_classifier["class"] == "Boa").sum()
df_password_classifier_MTB = df_password_classifier[df_password_classifier['class'].str.startswith("Muito Boa")]
df_password_classifier_Fraca = df_password_classifier[df_password_classifier['class'].str.startswith("Fraca")]
df_password_classifier_Ruim = df_password_classifier[df_password_classifier['class'].str.startswith("Ruim")]
df_password_classifier_MuitoRuim = df_password_classifier[df_password_classifier['class'].str.startswith("Muito Ruim")]
print(f"Num senhas Boas no aquivo do aluno: {df_password_classifier_Boas:,}")
# print(df_passwords_classifier_Boas)
print("Num senhas Muito Boas no arquivo do aluno: {:,}".format(len(df_password_classifier_MTB)))
print("Num senhas Fracas no arquivo do aluno: {:,}".format(len(df_password_classifier_Fraca)))
print("Num senhas Ruim no arquivo do aluno: {:,}".format(len(df_password_classifier_Ruim)))
print("Num senhas Muito Ruim no arquivo do aluno: {:,}".format(len(df_password_classifier_MuitoRuim)))


Comparativo qualitativo dos resultados

In [ ]:
df_password_classifier.info()

In [ ]:
df_password_classifier_Fraca['password']

In [ ]:
df_senhas_Fraca['password']

Comparando senhas fracas

In [9]:
conjunto1 = set(df_senhas_Fraca['password'])
conjunto2 = set(df_password_classifier_Fraca['password'])
# Diferença simétrica (elementos exclusivos em ambas as séries)
diferenca_simetrica = conjunto1.symmetric_difference(conjunto2)
print(diferenca_simetrica)

{'uas26', 'I\x11', '9\x16/\x1e', 'm.f', 'ZANA!', '1985m', 'kornÚl', '88jo', 'pea17', 'gÚdodo', 'olivÚr', '5X<)h', 'niu.', 'eukaþo', 'dßniel', 'k66', 'rm4ru', 'makßmi', '1(R', 'b@C', 'qsx]', 'R618t$', 'otto1', 'r8', '57r82', '9ol0p', 'too76', 'fs42%', '71@', 'M1rd@', 'ath21', 'T0rO@', 'n8bG', '\x1cP|', '12m3e', '\x04b\x0e)|', 'ebt96', 'mÇÇtmÇ', ')\x19Ih', 'llÚÚßß', '2@fm', 'gr123', 'z z z', '2}', 'tÚrÚrÚ', 'm3hdt', 'C\x05tJ', 'gne52', 'Y0 yo', 'ea123', 'v.+mu', 'x3l+B', 'X{}_|', 'l1', 'tr929', 'mri91', '%$$N', 'grbz4', '14ika', 'gnTÂ—', 'n4pl', 'k-zap', 'askimî', 'Trm94', 'indißn', 'z=0', 'Knd0r', 'AL12-', 'wsg14', '0if', 'pi123', 'ab&&', 'cari1', 'yese9', '122C', 'altÄ±', 'smk1', 'km982', 'nh989', 'zl123', 'tsc&w', 'loko2', 'zuiop³', '07Gl', '18aos', 'ab&Ú&', 'gk123', 'd]h', 'B(\x12', 'jo4vu', 'nuez.', '24Aos', 'sti11', 'H73T7', 'ayÅŸem', 'i9wj', '8432ÓÞ', 'epq$d', 'abcþde', '0p', 'yu5uf', '\x05xU]\x10', 'v9888', 'xX=a@', 'll12', 'T0bO@', '1g23l', 'su&i', '0^C', 'gle27', 'Go\x06', 'a12

In [11]:
# Diferença (elementos exclusivos em s1)
diferenca_s1 = conjunto1.difference(conjunto2)
print(diferenca_s1)

set()


In [12]:
# Diferença (elementos exclusivos em s2)
diferenca_s2 = conjunto2.difference(conjunto1)
print(diferenca_s2)

{'pea17', '07Gl', 'suy87', 'tÚrÚrÚ', 'ea123', 'nia10', 'cse21', '9ol0p', '8432ÓÞ', 'mÇÇtmÇ', 'gle27', '1985m', 'R618t$', '14ika', 'tr929', 'epq$d', 'gven6', 'X{}_|', 'm3hdt', 'tes¾ka', 'loko2', 'ayam_', 'll12', '123yx', 'fg95@', 'Y0 yo', 'T0bO@', 'jelsz¾', 'sti11', '88jo', 'yu5uf', '\x12px', '122C', 'z z z', 'tsc&w', 'eukaþo', 'r8', 'n8bG', 'askimî', 'qsx]', '57r82', 'ath21', '0^C', 'm2n1b', 'v9888', 'k66', 'H73T7', 'gr123', '0if', '2\x02Ng', 'smk1', '9\x16/\x1e', 'po09?', 'Knd0r', '+W\x18', 'm.f', 'B(\x12', '1g23l', 'zl123', 'olivÚr', 'p8', 'Go\x06', 'dßniel', 'rakÚta', 'qaz[', '210 9', 'ab&&', '71@', 'x3l+B', 'v.+mu', 'vk=u', '10aos', '18aos', 'd]h', '\x1cP|', '1(R', '2}', 'Trm94', '\x05xU]\x10', 'mri91', 'GoÉõ\x06', 'fs42%', 'AL12-', ')\x19Ih', 'ebt96', 'indißn', '2@fm', 'jo4vu', '%$$N', 'uas26', 'zuiop³', 'vit23', 'kev77', '12m3e', '*3P', 'T0rO@', 'abcþde', 'makßmi', 'D&^', 'I\x11', 'C\x05tJ', '5X<)h', '\x04b\x0e)|', 'ab&Ú&', 'pia89', 'km982', 'b@C', 'pi123', 'mh\x17', 'wsg14', 'zl

In [ ]:
pd.testing.assert_series_equal(df_senhas_Fraca['password'], df_password_classifier_Fraca['password'], check_names=False)

# Ordenações

Ordenar o arquivo completo de senhas pelo campo length em ordem decrescente.

In [ ]:
# ler o dataframe ordenado
# pegar o campo length e transformar para lsita df[coluna].tolist()
# Verificar se estar ordenado df[coluna].tolist() == sorted(df[coluna].tolist())